In [1]:
from nltk.tokenize import wordpunct_tokenize, WordPunctTokenizer
from typing import List

from flair.data import Sentence, Token
from flair.models import SequenceTagger
import re

In [2]:
flair_model = SequenceTagger.load('/nfs/gns/literature/Santosh_Tirunagari/GitHub/flair_models/ner/manual_annotated_dataset/only_flair_embeddings/best-model.pt')

2019-12-05 18:20:55,582 loading file /nfs/gns/literature/Santosh_Tirunagari/GitHub/flair_models/ner/manual_annotated_dataset/only_flair_embeddings/best-model.pt


In [3]:
def custom_tokenizer(text: str) -> List[Token]:
    """
    Tokenizer based on word and punctuations only.
    """
    tokens: List[Token] = []

    tokenizer = WordPunctTokenizer()

    text = tokenizer.tokenize(text)

    index = 0
    for index, word in enumerate(text):
        tokens.append(
            Token(
                text=word, start_position=index, whitespace_after=False
            )
        )

    return tokens


In [101]:
already_processed_files = sorted(glob.glob(result_json_dump_path + '*.json*'))
names = [os.path.basename(x) for x in already_processed_files]
already_processed_PMC_ids = [x[:-5]+'.json' for x in names]
already_processed_PMC_ids

result_file_name, already_processed_PMC_ids[0]

('PMC3549600_sentences.json', 'PMC1090599_sentences.json')

In [108]:
import time
import glob
import os
import json

article_path = '/nfs/gns/literature/machine-learning/evaluation/time_complexity/articles/'
all_files = sorted(glob.glob(article_path + '*.txt*'))
result_json_dump_path = '/nfs/gns/literature/machine-learning/evaluation/time_complexity/flair/'

offset = 26
article_file_path = article_path+'PMC3649237_sentences.txt'
with open(article_file_path, 'r') as f:
    article_contents = f.read()
    
article_contents.replace('\n','')    

"Using Information and Communication Technology in Home Care for Communication between Patients, Family Members, and Healthcare Professionals: A Systematic Review Academic Editor: Carlos De Las Cuevas Introduction. Information and communication technology (ICT) are becoming a natural part in healthcare both for delivering and giving accessibility to healthcare for people with chronic illness living at home. Aim. The aim was to review existing studies describing the use of ICT in home care for communication between patients, family members, and healthcare professionals. Methods. A review of studies was conducted that identified 1,276 studies. A selection process and quality appraisal were conducted, which finally resulted in 107 studies. Results. The general results offer an overview of characteristics of studies describing the use of ICT applications in home care and are summarized in areas including study approach, quality appraisal, publications data, terminology used for defining th

In [10]:
sentence = article_contents.replace('\n','')
Sentence(sentence, use_tokenizer=custom_tokenizer)
predicted_sentences = flair_model.predict(sentence)
predicted_sentences

AttributeError: 'str' object has no attribute 'tokens'

In [15]:
import time
import glob
import os
import json

article_path = '/nfs/gns/literature/machine-learning/evaluation/time_complexity/articles/'
all_files = sorted(glob.glob(article_path + '*.txt*'))
result_json_dump_path = '/nfs/gns/literature/machine-learning/evaluation/time_complexity/flair/'

offset = 26

already_processed_files = sorted(glob.glob(result_json_dump_path + '*.json*'))
names = [os.path.basename(x) for x in already_processed_files]
already_processed_PMC_ids = [x[:-5]+'.json' for x in names]

# article_file_path = article_path+'PMC3649237_sentences.txt'

for article_file_path in all_files:
    
    
    result_file_name = os.path.basename(article_file_path)[:-4] + '.json'
    
    if not result_file_name in already_processed_PMC_ids:
        with open(article_file_path, 'r') as f:
            article_contents = f.read()

            tstart = time.time()
            sentences = []
            for each_sentence in article_contents.split('\n'):
                sentences.append(Sentence(each_sentence, use_tokenizer=custom_tokenizer)) 

#             predicted_sentences = flair_model.predict(sentences)
            predicted_sentences = flair_model.predict(sentences, mini_batch_size=16)
            entity_dict={}
            all_entities = []
            list_names = ['exact', 'prefix','postfix','entity','ground']

            for i in range(0,len(sentences)):
                entities = predicted_sentences[i].to_dict(tag_type='ner')['entities']
                if entities:
                    tagged_sents = predicted_sentences[i].to_dict(tag_type='ner')
                    text = tagged_sents['text']
                    for root_node in tagged_sents['entities']:
                        exact = root_node['text']
                        prefix = text[root_node['start_pos']-offset:root_node['start_pos']-1]
                        postfix = text[root_node['end_pos']+1:root_node['end_pos']+offset]
                        entity = root_node['type']
                        normalise = ground_annotation(exact,entity)
                        data_names = [exact, prefix,postfix,entity,normalise]

                        all_entities.append(dict(zip(list_names,data_names)))

            tend = time.time()
            t_elapsed = round(tend - tstart)
            print(t_elapsed) 

            entity_dict['tagged_entities'] = all_entities
            entity_dict['time_taken'] = t_elapsed

            with open(result_json_dump_path+result_file_name, 'w') as outfile:
                json.dump(entity_dict, outfile)


   

2019-12-05 18:28:23,131 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-05 18:28:23,133 Ignore 87 sentence(s) with no tokens.
12
2019-12-05 18:28:34,796 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-05 18:28:34,798 Ignore 12 sentence(s) with no tokens.
9
2019-12-05 18:28:43,586 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-05 18:28:43,588 Ignore 2 sentence(s) with no tokens.
17
2019-12-05 18:29:01,087 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-05 18:29:01,089 Ignore 11 sentence(s) with no tokens.
9
2019-12-05 18:29:09,947 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-05 18:29:09,949 Ignore 13 sentence(s) with no tokens.
13
2019-12-05 18:29:23,310 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-05 18:29:23,312 Ignore 2 sentence(s) wit

2019-12-05 18:39:24,073 Ignore 9 sentence(s) with no tokens.
22
2019-12-05 18:39:45,650 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-05 18:39:45,652 Ignore 1 sentence(s) with no tokens.
12
2019-12-05 18:39:57,274 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-05 18:39:57,276 Ignore 7 sentence(s) with no tokens.
8
2019-12-05 18:40:05,201 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-05 18:40:05,203 Ignore 2 sentence(s) with no tokens.
11
2019-12-05 18:40:16,515 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-05 18:40:16,518 Ignore 15 sentence(s) with no tokens.
12
2019-12-05 18:40:28,776 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-05 18:40:28,779 Ignore 1 sentence(s) with no tokens.
0
2019-12-05 18:40:29,108 ACHTUNG: An empty Sentence was created! Are there empty strings in y

2019-12-05 18:48:13,137 Ignore 1 sentence(s) with no tokens.
16
2019-12-05 18:48:28,529 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-05 18:48:28,531 Ignore 9 sentence(s) with no tokens.
8
2019-12-05 18:48:36,868 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-05 18:48:36,870 Ignore 3 sentence(s) with no tokens.
14
2019-12-05 18:48:51,296 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-05 18:48:51,299 Ignore 1 sentence(s) with no tokens.
11
2019-12-05 18:49:01,533 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-05 18:49:01,535 Ignore 4 sentence(s) with no tokens.
7
2019-12-05 18:49:08,463 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-05 18:49:08,465 Ignore 10 sentence(s) with no tokens.
11
2019-12-05 18:49:18,959 ACHTUNG: An empty Sentence was created! Are there empty strings in y

2019-12-05 18:58:53,436 Ignore 1 sentence(s) with no tokens.
22
2019-12-05 18:59:15,228 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-05 18:59:15,230 Ignore 3 sentence(s) with no tokens.
28
2019-12-05 18:59:42,745 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-05 18:59:42,748 Ignore 1 sentence(s) with no tokens.
10
2019-12-05 18:59:53,084 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-05 18:59:53,086 Ignore 1 sentence(s) with no tokens.
17
2019-12-05 19:00:09,760 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-05 19:00:09,762 Ignore 1 sentence(s) with no tokens.
10
2019-12-05 19:00:20,282 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-05 19:00:20,284 Ignore 2 sentence(s) with no tokens.
7
2019-12-05 19:00:27,504 ACHTUNG: An empty Sentence was created! Are there empty strings in y

2019-12-05 19:10:42,880 Ignore 1 sentence(s) with no tokens.
16
2019-12-05 19:10:59,201 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-05 19:10:59,203 Ignore 7 sentence(s) with no tokens.
29
2019-12-05 19:11:28,474 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-05 19:11:28,477 Ignore 1 sentence(s) with no tokens.
13
2019-12-05 19:11:41,433 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-05 19:11:41,435 Ignore 9 sentence(s) with no tokens.
8
2019-12-05 19:11:49,254 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-05 19:11:49,256 Ignore 1 sentence(s) with no tokens.
23
2019-12-05 19:12:12,199 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-05 19:12:12,201 Ignore 1 sentence(s) with no tokens.
9
2019-12-05 19:12:20,734 ACHTUNG: An empty Sentence was created! Are there empty strings in yo

...

2019-12-06 02:43:51,369 Ignore 1 sentence(s) with no tokens.
45
2019-12-06 02:44:36,465 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-06 02:44:36,467 Ignore 1 sentence(s) with no tokens.
11
2019-12-06 02:44:47,773 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-06 02:44:47,775 Ignore 1 sentence(s) with no tokens.
10
2019-12-06 02:44:57,798 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-06 02:44:57,800 Ignore 1 sentence(s) with no tokens.
92
2019-12-06 02:46:29,394 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-06 02:46:29,396 Ignore 1 sentence(s) with no tokens.
30
2019-12-06 02:46:59,339 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-06 02:46:59,341 Ignore 1 sentence(s) with no tokens.
28
2019-12-06 02:47:27,645 ACHTUNG: An empty Sentence was created! Are there empty strings in 

2019-12-06 03:10:28,842 Ignore 7 sentence(s) with no tokens.
16
2019-12-06 03:10:44,687 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-06 03:10:44,689 Ignore 214 sentence(s) with no tokens.
11
2019-12-06 03:10:55,658 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-06 03:10:55,660 Ignore 1 sentence(s) with no tokens.
25
2019-12-06 03:11:20,601 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-06 03:11:20,603 Ignore 1 sentence(s) with no tokens.
28
2019-12-06 03:11:48,830 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-06 03:11:48,832 Ignore 1 sentence(s) with no tokens.
17
2019-12-06 03:12:06,289 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-06 03:12:06,291 Ignore 1 sentence(s) with no tokens.
15
2019-12-06 03:12:21,225 ACHTUNG: An empty Sentence was created! Are there empty strings i

2019-12-06 03:31:44,815 Ignore 1 sentence(s) with no tokens.
56
2019-12-06 03:32:40,590 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-06 03:32:40,592 Ignore 4 sentence(s) with no tokens.
41
2019-12-06 03:33:21,200 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-06 03:33:21,202 Ignore 1 sentence(s) with no tokens.
31
2019-12-06 03:33:52,321 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-06 03:33:52,323 Ignore 1 sentence(s) with no tokens.
7
2019-12-06 03:33:59,242 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-06 03:33:59,245 Ignore 1 sentence(s) with no tokens.
39
2019-12-06 03:34:38,790 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-06 03:34:38,792 Ignore 1 sentence(s) with no tokens.
16
2019-12-06 03:34:54,274 ACHTUNG: An empty Sentence was created! Are there empty strings in y

2019-12-06 03:54:28,604 Ignore 1 sentence(s) with no tokens.
21
2019-12-06 03:54:50,159 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-06 03:54:50,162 Ignore 1 sentence(s) with no tokens.
24
2019-12-06 03:55:14,629 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-06 03:55:14,630 Ignore 1 sentence(s) with no tokens.
30
2019-12-06 03:55:44,978 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-06 03:55:44,980 Ignore 1 sentence(s) with no tokens.
45
2019-12-06 03:56:30,405 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-06 03:56:30,407 Ignore 1 sentence(s) with no tokens.
16
2019-12-06 03:56:46,713 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-06 03:56:46,715 Ignore 1 sentence(s) with no tokens.
9
2019-12-06 03:56:55,550 ACHTUNG: An empty Sentence was created! Are there empty strings in y

2019-12-06 04:15:03,616 Ignore 1 sentence(s) with no tokens.
19
2019-12-06 04:15:23,069 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-06 04:15:23,071 Ignore 1 sentence(s) with no tokens.
17
2019-12-06 04:15:39,843 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-06 04:15:39,845 Ignore 1 sentence(s) with no tokens.
52
2019-12-06 04:16:31,266 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-06 04:16:31,268 Ignore 1 sentence(s) with no tokens.
14
2019-12-06 04:16:44,797 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-06 04:16:44,800 Ignore 1 sentence(s) with no tokens.
6
2019-12-06 04:16:51,133 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2019-12-06 04:16:51,135 Ignore 1 sentence(s) with no tokens.
54
2019-12-06 04:17:45,255 ACHTUNG: An empty Sentence was created! Are there empty strings in y

In [91]:
entity_dict

{'tagged_entities': [{'exact': 'chronic illnesses',
   'prefix': 'ead to people living with',
   'postfix': 'gaining control of their ',
   'entity': 'DS',
   'ground': 'http://www.ebi.ac.uk/efo/EFO_0009801'},
  {'exact': 'chronically ill',
   'prefix': 'of older [9] and severely',
   'postfix': 'people [10].',
   'entity': 'DS',
   'ground': 'http://www.ebi.ac.uk/efo/EFO_0007348'},
  {'exact': 'lung diseases',
   'prefix': 'es studied were heart and',
   'postfix': ' chronic wounds, diabetes',
   'entity': 'DS',
   'ground': 'http://www.ebi.ac.uk/efo/EFO_0003818'},
  {'exact': 'diabetes',
   'prefix': 'diseases, chronic wounds,',
   'postfix': ' cancer, and stroke.',
   'entity': 'DS',
   'ground': 'http://www.ebi.ac.uk/efo/EFO_0000400'},
  {'exact': 'cancer',
   'prefix': 'chronic wounds, diabetes,',
   'postfix': ' and stroke.',
   'entity': 'DS',
   'ground': 'http://www.ebi.ac.uk/efo/EFO_0000311'},
  {'exact': 'stroke',
   'prefix': 'ds, diabetes, cancer, and',
   'postfix': '',
 

In [16]:
4 - 284/27
8 - 182/18
16 - 128/19
32 - 109/136/13
128 - 152

nothing - 134/14

In [14]:
import requests

def ground_annotation(exact,entity):
    grounded_value = 'not-grounded'
    
    root_url = 'https://www.ebi.ac.uk/ols/api/search?'
    URL = root_url
    
    if entity == 'DS':
        URL = root_url+'q='+exact+'&exact=false&ontology=efo'
    elif entity == 'OG':
        URL = root_url+'q='+exact+'&exact=false&ontology=ncbitaxon'
    elif entity == 'GP':
        URL = 'http://10.7.35.118:8157/solr/Genes/select?q='+exact
    else:
#         print(URL)
        return 'URL missing'
    
    r = requests.get(URL)

    if r.status_code ==200:
        json_data = r.json()
    else:
        return grounded_value


    if json_data['response']['numFound']!=0:
        if entity in ['DS', 'OG']:
            grounded_value = json_data['response']['docs'][0]['iri']
        elif entity in ['GP']:
            grounded_value = json_data['response']['docs'][0]['ID']
    
    return grounded_value    